# Redis
### Store cars data

#### Install needed to run Redis

In [ ]:
!pip install redis

In [ ]:
#!pip install flask-bootstrap

#### Preparing the data

In [1]:
import pandas as pd
import pickle

In [2]:
df_carList = pd.read_csv("./datasets/Car_listings.csv", delimiter=",")

We can see that this dataset has a Price column, which we don't need, beacuse this is a rental company where customers doesn't buy the cars. Because of this we remove the Price column:

In [3]:
df_carList = df_carList.drop(['Price'], axis = 1)
df_carList.head(1)

,Year,Mileage,City,State,Vin,Make,Model
0,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed


In [4]:
df_carList = df_carList.sample(frac = 1)

#### We need to convert the csv file to JSON, so that we can store the data in redis

In [5]:
df_carList.to_json('./datasets/cars.json', orient='split', compression='infer', index='true')

In [6]:
df = pd.read_json('./datasets/cars.json', orient ='split', compression = 'infer')

In [ ]:
df

#### Redis

In [7]:
import redis
import json
import time

In [8]:
r = redis.Redis(db=1)
#r.flushdb()

In [10]:
with open('./datasets/cars.json') as json_file:
    data = json.load(json_file)
 
    # Print the type of data variable
    #print("Type:", type(data))
    
    #print("ITEMS: ", data.items())
      
    # Print the data of dictionary
    #print("\nData:", data['data'][0]['ID IPEDS Occupation Parent'])
    print("\nData:", data['data'][0])


Data: [2004, 38235, 'Marietta', ' GA', 'WDBSK75FX4F069367', 'Mercedes-Benz', 'SL-ClassSL500']


In [ ]:
# Loop og give hver værdi en key som er deres kolonne navn
columns = list(df)
listOfDict = []

def smallDictFunction(co: [], di: {}) -> {}:
    smallDicts = {}
    for c in co:
        for d in di:
            if (columns.index(c) == dic.index(d)):
                #print(c + ': ' + str(d))
                #print(d)
                smallDicts[c] = d
        
    return smallDicts
   

for dic in data['data']:
    listOfDict.append(smallDictFunction(columns, dic))
    if (data['data'].index(dic)+1 == 200000):
        break    # break here


#print(test())
#print(smallDicts)
print(listOfDict)
#print(fullDict)

In [ ]:
index_list = [*range(1, (len(listOfDict)+1), 1)]
len(index_list)

In [ ]:
zipped = zip(index_list, listOfDict)
fullDict = dict(zipped)
print(fullDict)

In [ ]:
with open("./datasets/carsDict.json", "w") as write_file:
    json.dump(fullDict, write_file, indent=4)

In [ ]:
with open('./datasets/carsDict.json') as json_file:
    data = json.load(json_file)

In [ ]:
for dic in data:
    p_mydict = pickle.dumps(data[dic])
    r.set(dic,p_mydict)

In [11]:
read_dict = r.get(200000)
yourdict = pickle.loads(read_dict)
yourdict

{'Year': 2012,
 'Mileage': 104716,
 'City': 'Tempe',
 'State': ' AZ',
 'Vin': '1HGCP2F67CA001727',
 'Make': 'Honda',
 'Model': 'Accord'}

In [ ]:
%%writefile redisSetup.py

import redis
import json
import pickle

def redisSimpleConnection():
    r = redis.Redis(db=1)
    return r
    
def redisSetup(df):
    r = redis.Redis(db=1)
    #r.flushdb()

    with open('./datasets/cars.json') as json_file:
        data = json.load(json_file)

    # Loop og give hver værdi en key som er deres kolonne navn
    columns = list(df)
    listOfDict = []

    def smallDictFunction(co: [], di: {}) -> {}:
        smallDicts = {}
        for c in co:
            for d in di:
                if (columns.index(c) == dic.index(d)):
                    smallDicts[c] = d

        return smallDicts


    for dic in data['data']:
        listOfDict.append(smallDictFunction(columns, dic))
        if (data['data'].index(dic)+1 == 200000):
            break    # break here

    index_list = [*range(1, (len(listOfDict)+1), 1)]
    len(index_list)


    zipped = zip(index_list, listOfDict)
    fullDict = dict(zipped)

    with open("./datasets/carsDict.json", "w") as write_file:
        json.dump(fullDict, write_file, indent=4)

    with open('./datasets/carsDict.json') as json_file:
        data = json.load(json_file)

    for dic in data:
        p_mydict = pickle.dumps(data[dic])
        r.set(dic,p_mydict)

    read_dict = r.get(200000)
    yourdict = pickle.loads(read_dict)

In [ ]:
!python redisSetup.py

In [ ]:
from redisFile import redisConnection
#redisConnection(df)

#### Postgres
Vi skal kunne søge i postgres efter en person, lave den person inclussiv addresse om til dict og smide den i mongo i samme dict som redis

In [ ]:
!pip install psycopg2

In [ ]:
from configparser import ConfigParser
import psycopg2

In [ ]:
def config(filename='../Postgresql/postgresDB.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [ ]:

def createDict(infoTuple, columnList, index):
    infoList = []
    for i in infoTuple:
        infoList.append(i)
        if (infoTuple.index(i) == index):
            break;
    return(dict(zip(columnList, infoList)))

fullUserDict= {}
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	# execute a statement
        print('Retrieving data:')
        print('--------------')
        #Retrieving data
        cur.execute('SELECT * from users where user_id = '+str(208))
        user = cur.fetchone()
        cur.execute('SELECT role from roles where role_id = '+str(user[6]))
        role = cur.fetchone()
        # Saving the user as a dict
        userTableColumns = ['user_id', 'firstName', 'lastName', 'age', 'email']
        userDict = createDict(user, userTableColumns, 4)
        userDict["role"] = role[0]
        print(userDict)
        print('--------------')
        
        cur.execute('SELECT address_id from users where user_id = '+str(208))
        address_id = cur.fetchone()
        cur.execute('SELECT * from address where address_id = '+str(address_id[0]))
        address = cur.fetchone()
        zipcode = address[2]
        cur.execute('SELECT * from cities where zipcode = '+str(zipcode))
        city = cur.fetchone()
        addressList = list(address)
        addressList.append(city[1])
        
        addressTableColumns = ['address_id', 'address', 'zipcode', 'city']
        addressDict = createDict(addressList, addressTableColumns, 3)
        print(addressDict)
        print('--------------')
        
        fullUserDict = userDict
        fullUserDict['address'] = addressDict
        
       
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
            #print(fullUserDict)
            return fullUserDict
            


if __name__ == '__main__':
    connect()
    
fullUserDict = connect()
print(fullUserDict)

In [43]:
%%writefile postgresSetup.py


from configparser import ConfigParser
import psycopg2

def config(filename='../Postgresql/postgresDB.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db


def createDict(infoTuple, columnList, index):
    infoList = []
    for i in infoTuple:
        infoList.append(i)
        if (infoTuple.index(i) == index):
            break;
    return(dict(zip(columnList, infoList)))

fullUserDict= {}

def retrieveUserInfo(uId, cur):
    #Retrieving data
    cur.execute('SELECT * from users where user_id = '+str(uId))
    user = cur.fetchone()
    cur.execute('SELECT role from roles where role_id = '+str(user[6]))
    role = cur.fetchone()
    # Saving the user as a dict
    userTableColumns = ['user_id', 'firstName', 'lastName', 'age', 'email']
    userDict = createDict(user, userTableColumns, 4)
    userDict["role"] = role[0]
    
    return userDict

def retrieveAddressInfo(uId, cur):
    cur.execute('SELECT address_id from users where user_id = '+str(uId))
    address_id = cur.fetchone()
    cur.execute('SELECT * from address where address_id = '+str(address_id[0]))
    address = cur.fetchone()
    zipcode = address[2]
    cur.execute('SELECT * from cities where zipcode = '+str(zipcode))
    city = cur.fetchone()
    addressList = list(address)
    addressList.append(city[1])
    
    addressTableColumns = ['address_id', 'address', 'zipcode', 'city']
    addressDict = createDict(addressList, addressTableColumns, 3)

    return addressDict

def connect(uId):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	# execute a statement
        addressDict = retrieveAddressInfo(uId, cur)

        fullUserDict = retrieveUserInfo(uId, cur)
        fullUserDict['address'] = addressDict
        
       
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
            #print(fullUserDict)
            return fullUserDict
            


if __name__ == '__main__':
    connect()



Overwriting postgresSetup.py


In [ ]:
from postgresSetup import connect
#connect()

#### MongoDB

In [ ]:
!pip install pymongo

In [ ]:
import pymongo
import bson
import pickle
import base64

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

#use database named "organisation"
mydb = myclient["ExamProject"]

#use collection named "developers"
mycol = mydb["Temporary"]

In [ ]:
def getData(n):
    dataList = []
    tic = time.perf_counter()
    d = r.get(n)
    yourdict = pickle.loads(d)
    dataList.append(yourdict)
    toc = time.perf_counter()
    dataList.append(toc - tic)
    return dataList

car = getData(1)
car[0]

In [ ]:
booking = {}
booking['user'] = fullUserDict
booking['car'] = car[0]

print(booking)

In [ ]:
x = mycol.insert_one(booking)

In [ ]:
for x in mycol.find():
    print(x)
    print('------------------')

In [42]:
%%writefile mongoDBSetup.py


import pymongo
import pickle
import time


def mongoConnection(connection_string, database_name, collection_name):
    myclient = pymongo.MongoClient(connection_string)

    #use database named "organisation"
    mydb = myclient[database_name]

    #use collection named "developers"
    mycol = mydb[collection_name]

    return mycol

def getDataFromMongo(r, n):
    dataList = []
    tic = time.perf_counter()
    d = r.get(n)
    yourdict = pickle.loads(d)
    dataList.append(yourdict)
    toc = time.perf_counter()
    dataList.append(toc - tic)
    return dataList

def createBooking(mycol, user, car):
    booking = {}
    booking['user'] = user
    booking['car'] = car
    
    mycol.insert_one(booking)

def findBookings(mycol):
    bookingList = []
    for x in mycol.find():
        bookingList.append(x)
    print(bookingList)
    return bookingList

Overwriting mongoDBSetup.py


#### Neo4j

In [ ]:
!pip install neo4j

In [ ]:
import logging
import sys

from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable

In [ ]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password), database='analisys')

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    @staticmethod
    def enable_log(level, output_stream):
        handler = logging.StreamHandler(output_stream)
        handler.setLevel(level)
        logging.getLogger("neo4j").addHandler(handler)
        logging.getLogger("neo4j").setLevel(level)

    def create_friendship(self, user_id, 
                      firstName, 
                      lastName,
                      age,
                      email,
                      address,
                      zipcode,
                      city,
                      year,
                      mileage,
                      cityC,
                      state,
                      vin,
                      make,
                      model, books):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_and_return_friendship, user_id, 
                      firstName, 
                      lastName,
                      age,
                      email,
                      address,
                      zipcode,
                      city,
                      year,
                      mileage,
                      cityC,
                      state,
                      vin,
                      make,
                      model, books)
            for row in result:
                print("Created friendship between: {u}, {c} from {books}"
                      .format(
                          u=row['u'],
                          c=row['c'],
                          books=row['books']))

    @staticmethod
    def _create_and_return_friendship(tx, user_id, 
                      firstName, 
                      lastName,
                      age,
                      email,
                      address,
                      zipcode,
                      city,
                      year,
                      mileage,
                      cityC,
                      state,
                      vin,
                      make,
                      model, books):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "MERGE (u:User { user_id: $user_id, firstName: $firstName, lastName: $lastName, age: $age, email: $email, address: $address, zipcode: $zipcode, city: $city}) "
            "MERGE (c:Car { year: $year, mileage: $mileage, cityC: $cityC, state: $state, vin: $vin, make: $make, model: $model }) "
            "MERGE (u)-[k:BOOKS { from: $books }]->(c) "
            "RETURN u, c, k"
        )
        result = tx.run(query, user_id=user_id, firstName=firstName, lastName=lastName, age=age, email=email, 
                        address=address, zipcode=zipcode, city=city, year=year, mileage=mileage, cityC=cityC, state=state,
                        vin=vin, make=make, model=model, books=books)
        try:
            return [{
                        "u": row["u"]["user_id"],
                        "c": row["c"]["year"],
                        "books": row["k"]["books"]
                    }
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise


if __name__ == "__main__":
    bolt_url = "bolt://localhost:7687"
    user = "neo4j"
    password = "1234"
    App.enable_log(logging.INFO, sys.stdout)
    app = App(bolt_url, user, password)
    
    infoList = []
    for x in mycol.find():
        #print(list(x.values())[1])
        for key, values in x.items():
            if (key == 'user'):
                #x = thisdict["model"]
                listVU = list(values)
                #print(values)

                for k, v in values.items():
                    infoList.append(v)
                infoList.pop()
                infoList.pop()
                for k, v in values['address'].items():
                    infoList.append(v)
                del infoList[5]
                #print(listVU)
                print('--------------')
            if (key == 'car'):
                print(values)
                for k, v in values.items():
                    infoList.append(v)
            #print('key: ' + key)
            #print('values: ' + str(values))
        print(infoList)
        app.create_friendship(infoList[0], infoList[1], infoList[2], infoList[3], infoList[4], infoList[5], 
                              infoList[6], infoList[7], infoList[8], infoList[9], infoList[10], infoList[11], 
                              infoList[12], infoList[13], infoList[14], 'books')
        print('------------------')
        infoList = []
    
    #app.create_friendship(405, 'Belinda', 'Foster', 44, 'b.foster@randatmail.com', 'Kongshøj Allé 65', 4220, 'Korsør',
                         #2017, 15630, 'Katy', 'TX', '1FMJU1JT4HEA55066', 'Ford', 'ExpeditionXLT', 'books')
    #app.find_person("Alice")
    
    app.close()

In [41]:
%%writefile neo4jSetup.py

import logging
import sys

from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable


def close(driver):
    driver.close()

def connectNeo(uri, user, password, db):
    driver = GraphDatabase.driver(uri, auth=(user, password), database=db)
    return driver

def create_relationsship_books(tx, user_id, 
                      firstName, 
                      lastName,
                      age,
                      email,
                      address,
                      zipcode,
                      city,
                      year,
                      mileage,
                      cityC,
                      state,
                      vin,
                      make,
                      model, books):
    query = (
            "MERGE (u:User { user_id: $user_id, firstName: $firstName, lastName: $lastName, age: $age, email: $email, address: $address, zipcode: $zipcode, city: $city}) "
            "MERGE (c:Car { year: $year, mileage: $mileage, cityC: $cityC, state: $state, vin: $vin, make: $make, model: $model }) "
            "MERGE (u)-[k:BOOKS { from: $books }]->(c) "
            "RETURN u, c, k"
        )
    result = tx.run(query, user_id=user_id, firstName=firstName, lastName=lastName, age=age, email=email, 
                    address=address, zipcode=zipcode, city=city, year=year, mileage=mileage, cityC=cityC, state=state,
                    vin=vin, make=make, model=model, books=books)
    try:
        return [{
                    "u": row["u"]["user_id"],
                    "c": row["c"]["year"],
                    "books": row["k"]["books"]
                }
                for row in result]
    # Capture any errors along with the query and data for traceability
    except ServiceUnavailable as exception:
        logging.error("{query} raised an error: \n {exception}".format(
            query=query, exception=exception))
        raise

def createNodes(driver, user_id, 
                      firstName, 
                      lastName,
                      age,
                      email,
                      address,
                      zipcode,
                      city,
                      year,
                      mileage,
                      cityC,
                      state,
                      vin,
                      make,
                      model, books):
    with driver.session() as session:
        session.write_transaction(create_relationsship_books, user_id, 
                      firstName, 
                      lastName,
                      age,
                      email,
                      address,
                      zipcode,
                      city,
                      year,
                      mileage,
                      cityC,
                      state,
                      vin,
                      make,
                      model, books)

def createAnalysis(mycol, driver):
    infoList = []
    for x in mycol.find():
        for key, values in x.items():
            if (key == 'user'):
                for k, v in values.items():
                    infoList.append(v)
                infoList.pop()
                infoList.pop()
                for k, v in values['address'].items():
                    infoList.append(v)
                del infoList[5]
            if (key == 'car'):
                for k, v in values.items():
                    infoList.append(v)

        createNodes(driver, infoList[0], infoList[1], infoList[2], infoList[3], infoList[4], infoList[5], 
                                infoList[6], infoList[7], infoList[8], infoList[9], infoList[10], infoList[11], 
                                infoList[12], infoList[13], infoList[14], 'books')

        infoList = []


Overwriting neo4jSetup.py


In [35]:
%%writefile functions.py

from flask import Flask, request, jsonify, render_template
from redisSetup import *
from postgresSetup import *
from mongoDBSetup import *
from neo4jSetup import *

# ---------- Redis -------------
r = redisSimpleConnection()
def getCar(n):
    read_dict = r.get(n)
    car = pickle.loads(read_dict)
    return car

# ---------- Postgres -------------
def getUser(n):
    user = connect(n)
    return user

# ---------- Mongo -------------
def createBookingLast(connection_string, database_name, collection_name, uId, cId):
    mycol = mongoConnection(connection_string, database_name, collection_name)
    user = getUser(uId)
    car = getCar(cId)
    createBooking(mycol, user, car)

def findBookingsLast(connection_string, database_name, collection_name):
    mycol = mongoConnection(connection_string, database_name, collection_name)
    return findBookings(mycol)

# ---------- Neo4j -------------
def createAnalysisLast(connection_string, database_name, collection_name):
    mycol = mongoConnection(connection_string, database_name, collection_name)
    bolt_url = "bolt://localhost:7687"
    user = "neo4j"
    password = "1234"
    db = 'analysis'
    
    driver = connectNeo(bolt_url, user, password, db)
    createAnalysis(mycol, driver)
    close(driver)
    return 'done'

Overwriting functions.py


In [36]:
!python functions.py

In [39]:
%%writefile application.py

from flask import Flask, request, jsonify, render_template
from functions import *
from threading import Timer

app = Flask(__name__, template_folder='hmtl_documents')

connection_string = "mongodb://localhost:27017/"
database_name = "ExamProject"
collection_name = "Bookings"

@app.route('/')
def bookingEndpoint():
    return render_template('booking.html')

@app.route('/booked', methods=['GET', 'POST'])
def booked():
    if request.method == 'POST':
        uId = request.form['uId']
        cId = request.form['cId']
        
        createBookingLast(connection_string, database_name, collection_name, uId, cId)
          
        return render_template("booked.html", data=[uId, cId])

@app.route('/admin')
def admin():
    bookings = findBookingsLast(connection_string, database_name, collection_name)
    
    return render_template('admin.html', bookings=bookings)

@app.route('/adminAnalysis', methods=['GET', 'POST'])
def adminAnalysis():
    if request.method == 'POST':
        createAnalysisLast(connection_string, database_name, collection_name)
        
        return render_template("adminAnalysis.html")
    
if __name__ == '__main__':
    app.run(debug=True)

Overwriting application.py


In [ ]:
!python application.py

http://localhost:5000/